In [25]:
import pandas as pd
from pprint import pprint
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.neighbors import KNeighborsClassifier

Reading and Understanding data

In [3]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
sample = pd.read_csv("sample_submission.csv")

In [9]:
pprint(vars(train))

{'_data': BlockManager
Items: Index(['id', 'keyword', 'location', 'text', 'target'], dtype='object')
Axis 1: RangeIndex(start=0, stop=7613, step=1)
IntBlock: slice(0, 8, 4), 2 x 7613, dtype: int64
ObjectBlock: slice(1, 4, 1), 3 x 7613, dtype: object,
 '_is_copy': None,
 '_item_cache': {}}


In [27]:
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [14]:
sample.head()

,id,target
0,0,0
1,2,0
2,3,0
3,9,0
4,11,0


### Try k-nearest neighbour on the data

In [15]:
trainer,valid = train_test_split(train,
                                                  random_state=42, 
                                                  test_size=0.2, shuffle=True)

In [13]:
help(train_test_split)

Help on function train_test_split in module sklearn.model_selection._split:

train_test_split(*arrays, **options)
    Split arrays or matrices into random train and test subsets
    
    Quick utility that wraps input validation and
    ``next(ShuffleSplit().split(X, y))`` and application to input data
    into a single call for splitting (and optionally subsampling) data in a
    oneliner.
    
    Read more in the :ref:`User Guide <cross_validation>`.
    
    Parameters
    ----------
    *arrays : sequence of indexables with same length / shape[0]
        Allowed inputs are lists, numpy arrays, scipy-sparse
        matrices or pandas dataframes.
    
    test_size : float, int or None, optional (default=None)
        If float, should be between 0.0 and 1.0 and represent the proportion
        of the dataset to include in the test split. If int, represents the
        absolute number of test samples. If None, the value is set to the
        complement of the train size. If ``train_s

In [19]:
trainer.shape

(6090, 5)

In [21]:
tfv = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), use_idf=1,smooth_idf=1,sublinear_tf=1,
            stop_words = 'english')

In [23]:
tfv.fit(list(trainer.text.values) + list(valid.text.values))

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=3, ngram_range=(1, 3), norm='l2', preprocessor=None,
                smooth_idf=1, stop_words='english', strip_accents='unicode',
                sublinear_tf=1, token_pattern='\\w{1,}', tokenizer=None,
                use_idf=1, vocabulary=None)

In [24]:
xtrain_tfv =  tfv.transform(trainer.text.values) 
xvalid_tfv = tfv.transform(valid.text.values)

In [28]:
clf = KNeighborsClassifier(n_neighbors=3)
clf.fit(xtrain_tfv, trainer.target)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='uniform')

In [29]:
print("Test set predictions:", clf.predict(xvalid_tfv))

Test set predictions: [0 0 0 ... 0 0 0]


In [30]:
print("Test set accuracy: {:.2f}".format(clf.score(xvalid_tfv, valid.target)))

Test set accuracy: 0.68
